# T-тест и распределение Стьюдента

Z-тест и T-тест используют для проверки гипотез о среднем значении, но при разных обстоятельствах. Z-тест используют, когда известно стандартное отклонение генеральной совокупности. T-тест же обычно используют, когда стандартное отклонение генеральной совокупности неизвестно. Чтобы провести Т-тест нужно: 
- познакомиться с ESE 
- узнать что такое число степеней свободы
- научиться считать t- статистику.

Важно помнить: 
- t-статистика живёт в распределении Стьюдента 
    
- z-статистика живёт в стандартном нормальном распределении

### Рассчет ESE

Поскольку ESE считается на основе стандартного отклоение нам нужна функция для подсчета стандартного отклонения и дисперсии

In [1]:
def sd_calculation_and_disp(x):
    mean = sum(x)/len(x)
    difference_sqare = [(i - mean)**2 for i in x]
    disp = sum(difference_sqare)/(len(difference_sqare)-1)
    sd = disp**0.5
    return disp, sd

Дальше можно посчитать ESE

<div style="border:2px solid black; padding:10px;">
  <p> Оценка стандартной ошибки (англ. Estimated Standard Error, ESE) — это стандартное отклонение выборочных средних, вычисленное на основе стандартного отклонения выборки.</p>
</div>

$$
ESE = \frac{s_X}{\sqrt{n}}.
$$


Напишем функцию для рассчета ESE

In [2]:
import math

In [3]:
def ESE_calculation(sd, n):
    result = sd/math.sqrt(n)
    return result

**Пример работы функции:** Вычислим оценку стандартной ошибки (ESE), зная размер выборки и стандартное отклонение.

In [4]:
sd = 12 # Стандартное отклонение
n = 256 # Размер выборки

In [5]:
result = ESE_calculation(sd, n)

In [6]:
result

0.75

### Рассчет числа степеней свободы (df)


<div style="border:2px solid black; padding:10px;">
  <p> Количество степеней свободы -  какое количество «независимых» значений может варьироваться при известном среднем.</p>
</div>

$$
df = n - 1.
$$


Распределение Стьюдента зависит от числа степеней свободы (косвенно от размера выборки). С увеличением размера выборки увеличивается и значение df, и t-распределение всё больше и больше приближается к стандартному нормальному распределению.

### Рассчет t-статистики


Наша новая t-статистика обитает в распределении Стьюдента. Оно определяется количеством степеней свободы, которое зависит от размера выборки, а также вместо стандартной ошибки SE, которая использовалась в z-тест, мы используем оценку стандартной ошибки по выборке ESE:

$$
t = \frac{\overline{x} - \mu}{ESE},
$$


Напишем функцию для рассчета t - статистики

In [7]:
def t_calculation(x, m, ESE):
    # где x - среднее по выборке
    # m - среднее генеральной совокупности
    # ESE - оценка стандартной ошибки
    t = (x-m)/ESE
    return t

**Пример работы функции:** Вы хотите проверить гипотезу,  что средняя масса мешков с кофе, поставляемых поставщиком, равна 50 кг. Среднее значение по этой выборке из 40 мешков: 50.2 кг. Стандартное отклонение: 0.5 кг.

In [8]:
x = 50.2
m = 50

In [9]:
result = t_calculation(x, m, ESE_calculation(0.5, 40))

In [10]:
result

2.5298221281347395

Теперь мы умеем счиать ESE, число степеней свободы и t-статистику. Можно приступать к проведению тестов.

Сценарий t-теста

1. Формулируем $H_0$ и $H_1$.
2. Выбираем $\alpha$.
3. Считаем выборочное среднее.
4. Считаем выборочное стандартное отклонение.
5. Считаем t-статистику.
6. Считаем количество степеней свободы по формуле:

   $df = n - 1$,

   где $n$ — количество наблюдений в выборке.
7. Считаем p-value.
8. Сравниваем p-value с уровнем значимости:
   - Если p-value меньше уровня значимости, принимаем решение, что надо отклонить нулевую гипотезу.
   - Если p-value больше уровня значимости, то оснований отклонять нулевую гипотезу нет.


Соберем все детали в одну, добавим сравнение с альфа и принятие решения в функцию

In [11]:
from scipy import stats

def t_calculation_decision(x, m, ESE, n, alpha=0.05):
    # где x - среднее по выборке
    # m - среднее генеральной совокупности
    # ESE - оценка стандартной ошибки
    
    t = (x-m)/ESE
    df = n-1
    p_value = stats.t.sf(abs(t), df) * 2
    decision = "Отвергаем нулевую гипотезу" if p_value < alpha else "Не отвергаем нулевую гипотезу"
    return t, p_value, decision

**Пример работы функции:** на задаче с кофейными мешками выше.<br>
**Гипотезы:**
- Нулевая гипотеза $H_0$: $\mu = 50$ (средняя масса мешков с кофе, поставляемых поставщиком, равна 50)
- Альтернативная гипотеза $H_1$: $\mu \neq 50$ (не равна 50)

In [12]:
result = t_calculation_decision(x, m, ESE_calculation(0.5, 40), 40)

In [13]:
result

(2.5298221281347395, 0.015563362453317516, 'Отвергаем нулевую гипотезу')

Так как p-value 0.015, что меньше уровня значимости в 0.05, у нас есть основания отвергнуть нулевую гипотезу. Значит, средняя масса мешка с кофе отличается от нашего предположения в 50 кг.

**<p>Задача</p>**
На производстве электрических самокатов нужно проверить гипотезу, что среднее время зарядки батареи самоката составляет 
3 часа. <br>Всего у нас 20 самокатов.<br>
Среднее значение по этой выборке: 3.1 часа.<br>
Стандартное отклонение: 0.15 часа.<br>

**Гипотезы:**
- Нулевая гипотеза $H_0$: $\mu = 3$ 
- Альтернативная гипотеза $H_1$: $\mu \neq 3$



In [14]:
result = t_calculation_decision(3.1, 3, ESE_calculation(0.15, 20), 20)

In [15]:
result

(2.9814239699997227, 0.007670613445788524, 'Отвергаем нулевую гипотезу')

Так как p-value 0.007 меньше уровня значимости 0.05 у нас есть основания отвергнуть нулевую гипотезу. Значит, среднее время зарядки батареи самоката статистически значимо отличается от нашего предположения в  3 часа.

# Двухвыборочные тесты

Помимо одновыборочных тестов, в статистике существуют двухвыборочные тесты. Такие тесты используются для проверки гипотез о равенстве средних по двум генеральным совокупностям. 

Сценарий для двухвыборочного абсолютно такой же, однако ключевые метрики считаются по другому.

$$
t = \frac{(\overline{x}_1 - \overline{x}_2) - (\mu_1 - \mu_2)}{ESE},
$$

где

$$
ESE = \sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}.
$$


В случае, если нулевая гипотеза сформулирована как: $$\mu_1 - \mu_2 = 0 $$ 

<br> то формула для подсчета t-статистики упрощается:

$$
t = \frac{\overline{x}_1 - \overline{x}_2}{ESE}
$$


Формула для подсчета числа степеней свободы будет выглядеть так:

$$
df = \frac{(n - 1) \cdot (s_1^2 + s_2^2)^2}{s_1^4 + s_2^4},
$$

где \( n \) — размер одной выборки.


Напишем функции для подсчета метрик с новыми формулами, и подставим для проверки туда условия из задачи:

**Задача** <br>

У нас в распоряжении оказались данные о количестве пятен для 300 северных и 300 южных жирафов. Среднее количество пятен для выборки северных жирафов оказалось равно 203, для южных — 198. Стандартное отклонение по выборке северных жирафов составило
35, по выборке южных — 33

Функция для подсчета ESE для двухвыборочного теста

In [16]:
def ESE_duo(s1, s2, n1, n2):
# Стандартное отклонение выборки первой s1
# Стандартное отклонение второй выборки s2
# Размер первой выборки n1
# Размер второй выборки n2
    ESE = math.sqrt(((s1**2)/n1)+((s2**2)/n2))
    return ESE

In [17]:
my_ESE = ESE_duo(35, 33, 300, 300)

In [18]:
my_ESE

2.777288845859093

Функция для подсчета t - статистики для двухвыборочного теста

In [19]:
def t_stat_duo(x1, x2, m1, m2, ESE):
#     Среднее первой выборки x1
#     Среднее второй выборки x2
#     Среднее количество пятен в первой генеральной совокупности m1
#     Среднее количество пятен во второй генеральной совокупности m2

    t = ((x1-x2) - (m1 - m2))/ ESE
    return t

In [20]:
my_t_stat = t_stat_duo(203, 198, 0, 0, my_ESE)

In [21]:
my_t_stat

1.8003168836597405

Функция подсчета числа степеней свободы для двухвыборочного теста

In [22]:
def df_count(n, s1, s2):
#     Размер выборок должен быть одинаковый n
#     Стандартное отклонение первой выборки s2
#     Стандартное отклонение второй выборки s1
    df = ((n-1)*(s1**2 + s2**2)**2)/((s1**4) + (s2**4))
    return df

In [23]:
my_df = df_count(300, 35, 33)

In [24]:
my_df

595.9414817390061

In [25]:
p_value = stats.t.sf(abs(my_t_stat), my_df) * 2

In [26]:
p_value

0.07231617285809387

Теперь объединим все формулы в одну функцию, добавим туда решение о принятии гипотезы

In [27]:
def t_stat_duo(x1, x2, m1, m2, n1, n2, s1, s2, alpha=0.05):
    """
    Функция для выполнения двухвыборочного t-теста с разными дисперсиями.

    x1, x2: Средние выборок
    m1, m2: Средние генеральных совокупностей
    n1, n2: Размеры выборок
    s1, s2: Стандартные отклонения выборок
    alpha: Уровень значимости
    """
    ESE = math.sqrt(((s1**2)/n1) + ((s2**2)/n2))
    t = ((x1 - x2) - (m1 - m2)) / ESE
    df = ((n1-1)*(s1**2 + s2**2)**2)/((s1**4) + (s2**4))
    p_value = stats.t.sf(abs(t), df) * 2
    decision = "Отвергаем нулевую гипотезу" if p_value < alpha else "Не отвергаем нулевую гипотезу"
    output = (
        f"Результаты t-теста:\n"
        f"t-статистика: {t:.3f}\n"
        f"Ошибка стандартной оценки (ESE): {ESE:.3f}\n"
        f"Степени свободы (df): {df:.3f}\n"
        f"p-значение: {p_value:.5f}\n"
        f"Решение: {decision}"
    )

    return print(output)

Применим функции при решении задачи

**Задача**

 Сравнив несколько пылесосов произведенных с нового и старого станка, инженеры обнаружили разницу в среднем количестве шпунтиков внутри пылесосов.  Есть ли статистически значимые различия в количестве шпунтиков у пылесосов, произведённых на двух станках? <br>
Вот данные, которые получили инженеры:

- Среднее количество шпунтиков у пылесосов с нового станка: $\bar{x}_1 = 1240$
- Среднее количество шпунтиков у пылесосов со старого станка: $\bar{x}_2 = 1203$
- Стандартное отклонение количества шпунтиков у пылесосов с нового станка: $s_1 = 314$
- Стандартное отклонение количества шпунтиков у пылесосов со старого станка: $s_2 = 319$
- Количество пылесосов в каждой выборке: $n = 600$

**Гипотезы:**
- Нулевая гипотеза $H_0$: $\mu_1 - \mu_2 = 0$ (нет различий в количестве шпунтиков)
- Альтернативная гипотеза $H_1$: $\mu_1 - \mu_2 \neq 0$ (есть статистически значимые различия)


In [28]:
x1, x2 = 1240, 1202 # Средние выборок
m1,m2 = 0, 0 # Средние генеральных совокупностей
n1, n2 = 600, 600 # Размеры выборок
s1,s2 = 314, 319 # Стандартные отклонения выборок

In [29]:
t_stat_duo(x1, x2, m1, m2, n1, n2, s1, s2, alpha = 0.05)

Результаты t-теста:
t-статистика: 2.079
Ошибка стандартной оценки (ESE): 18.274
Степени свободы (df): 1197.701
p-значение: 0.03778
Решение: Отвергаем нулевую гипотезу


Отвергаем нулевую гипотезу о равенстве средних двух выборок. Новый станок действительно закручивает в роботов-пылесосов больше шпунтиков, чем старый!

### Бакетный тест

В случаях когда описывается поведение людей, оно редко имеет вид нормального распределения.  Например, есть мобильное приложение, и мы измеряем активность пользователей на основе времени, проведенного в приложении. Изначальные данные о времени сеанса сильно скошены вправо, так как большинство пользователей проводит в приложении очень мало времени, в то время как малая часть пользователей проводит значительно больше времени.

Однако, Центральная предельная теорема (ЦПТ) гласит, что распределение средних значений многих независимых и одинаково распределенных случайных величин, независимо от формы исходного распределения, будет приближаться к нормальному распределению, если размер выборок достаточно велик.

Это позволяет применять статистические методы, предполагающие нормальность распределения данных, такие как t-тесты для сравнения средних, даже если исходные данные не были нормально распределены.

Допустим нам нужно сравнить посещение двух тренажерных залов. Из каждого зала есть 3000 наблюдений, которые нельзя использовать напрямую, из-за ненормального распределения. Разобьём эти наблюдения на 100 групп случайным образом. В каждой группе посчитаем среднее. Правда ли что в первый зал ходят больше чем во второй?

Выделив 100 бакетов для каждой группы, мы получили следующие данные:

- среднее количество посещений по 100 бакетам в «Мощном жиме»: $\bar{x}_1 = 30.3$,
- среднее количество посещений по 100 бакетам в «Заводном отжиме»: $\bar{x}_2 = 30.1$,
- стандартное отклонение по 100 бакетам в «Мощном жиме»: $s_1 = 0.7$,
- стандартное отклонение по 100 бакетам в «Заводном отжиме»: $s_2 = 0.5$,
- количество наблюдений $n = 100$ для обеих групп.

Сформулируем нулевую и альтернативную гипотезы и зафиксируем уровень значимости:

- $H_0: \mu_1 - \mu_2 = 0$,
- $H_1: \mu_1 - \mu_2 \neq 0$,
- Уровень значимости, $\alpha = 0.05$.


In [30]:
x1, x2 = 30.3, 30.1 # Средние выборок
m1,m2 = 0, 0 # Средние генеральных совокупностей
n1, n2 = 100, 100 # Размеры выборок
s1,s2 = 0.7, 0.5 # Стандартные отклонения выборок

Воспользуемся уже написанной формулой для t-теста для сравнения двух выборок

In [31]:
t_stat_duo(x1, x2, m1, m2, n1, n2, s1, s2, alpha = 0.05)

Результаты t-теста:
t-статистика: 2.325
Ошибка стандартной оценки (ESE): 0.086
Степени свободы (df): 179.155
p-значение: 0.02120
Решение: Отвергаем нулевую гипотезу


Значение соответствующего p-value 0.02. Это число меньше, чем уровень значимости 0.05,  значит, мы отвергаем 
нулевую гипотезу и считаем, что действительно есть статистически значимое различие, которое оценивается в 
30.3−30.1 = 0.2 дня. 

# Распределение Бернули

Иногда случайная величина может принимать лишь 2 значения, 0 или 1 Распределение такой случайной величины называют распределением Бернулли.

Ипользуется для случаев когда данные в виде: сдал зачет/не сдал, пришел покупатель/не пришел. Для проверки таких гипотез существует специальный тест, который называется **Z-тестом для пропорций**. 

$$
z = \frac{(\overline{p}_1 - \overline{p}_2) - (\overline{p}_1 - \overline{p}_2)}{ESE},
$$

где

$$
ESE = \sqrt{\frac{\overline{p}_1 \cdot (1 - \overline{p}_1)}{n_1} + \frac{\overline{p}_2 \cdot (1 - \overline{p}_2)}{n_2}}.
$$



Сценарий:

1. Формулируем $H_0, H_1$.
2. Выбираем уровень значимости $\alpha$.
3. Считаем разность выборочных средних, $\overline{p}_1 - \overline{p}_2$.
4. Считаем $ESE$.
5. Считаем z-статистику.
6. Считаем p-value.
7. Сравниваем p-value с уровнем значимости. Если p-value меньше уровня значимости, принимаем решение, что надо отклонить нулевую гипотезу.



Напишем функцию, которая поможет нам посчитать Z-тест для пропорций

In [32]:
def z_prop_test(p1, p2, n1, n2, alpha=0.05):
    """
    Функция для выполнения z-теста пропорций.

    p1, p2: Средние выборок
    n1, n2: Размеры выборок
    alpha: Уровень значимости
    """
    ESE = math.sqrt(((p1 * (1 - p1)) / n1) + ((p2 * (1 - p2) )/ n2))
    z = (p1 - p2) / ESE
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    decision = "Отвергаем нулевую гипотезу" if p_value < alpha else "Не отвергаем нулевую гипотезу"
    output = (
        f"Результаты z-теста:\n"
        f"z-статистика: {z:.3f}\n"
        f"Ошибка стандартной оценки (ESE): {ESE:.3f}\n"
        f"p-значение: {p_value:.5f}\n"
        f"Решение: {decision}"
    )

    return print(output)

**Пример работы функции на основе задачи:** В выборке северных жирафов доля веселившихся составила 0.7. Это будет наше значение р1. В выборке южных жирафов аналогичная доля составила 0.78 Это значение р2. Размер обеих выборок равен по 300 жирафам. 

In [33]:
z_prop_test(0.7, 0.78, 300, 300)

Результаты z-теста:
z-статистика: -2.243
Ошибка стандартной оценки (ESE): 0.036
p-значение: 0.02489
Решение: Отвергаем нулевую гипотезу


Мы получили p-value 0.024. Это значение меньше выбранного уровня значимости. Значит, можно сделать вывод о том, что доля веселившихся северных жирафов статистически значимо отличается от доли веселившихся южных жирафов. Более того, мы видим, что это отличие в меньшую сторону.

**Задача**


Ваш друг из Москвы высказал гипотезу о том, что доля жителей Москвы, знающих статистику, больше доли жителей Санкт-Петербурга, знающих статистику. Вы, будучи как раз таким жителем Санкт-Петербурга, подошли к делу основательно: провели в двух городах опрос и получили следующие результаты:

- доля респондентов в Москве, знающих статистику: $\overline{p}_1 = 0.1$,
- доля респондентов в Санкт-Петербурге, знающих статистику: $\overline{p}_2 = 0.11$,
- количество респондентов в Москве: $n_1 = 1345$,
- количество респондентов в Санкт-Петербурге: $n_2 = 1326$.

Проверьте гипотезу о том, что доля жителей Москвы, знающих статистику, не отличается от доли таких жителей в Санкт-Петербурге:

- $H_0: \overline{p}_1 - \overline{p}_2 = 0$,
- $H_1: \overline{p}_1 - \overline{p}_2 \neq 0$.




In [34]:
z_prop_test(0.11, 0.1, 1345, 1326)

Результаты z-теста:
z-статистика: 0.843
Ошибка стандартной оценки (ESE): 0.012
p-значение: 0.39914
Решение: Не отвергаем нулевую гипотезу


Соответствующее значение p-value 0.4. Так как это значение выше уровня значимости, мы не можем отвергнуть нулевую гипотезу о равенстве двух выборок, а значит количество людей знающих статистику, по нашим данным, в обоих городах одинаково

**Задача**

Технология приготовления пиццы с грибами предполагает, что в большой пицце будет в среднем на 13 кусочков грибов больше, чем в маленькой. Вы решили проверить, так ли это. Для этого вы посчитали количество кусочков в 70 больших пиццах и в 70 маленьких.

Вы получили такие результаты:

- среднее количество кусочков в больших пиццах: $ \bar{x}_1 = 61 $,
- среднее количество кусочков в маленьких пиццах: $ \bar{x}_2 = 49 $,
- стандартное отклонение количества кусочков в больших пиццах: $ s_1 = 5 $,
- стандартное отклонение количества кусочков в маленьких пиццах: $ s_2 = 5 $,
- количество пицц каждого размера: $ n = 70 $.

Проверьте гипотезу о том, что разница в количестве кусочков равна 13:

- $ H_0: \mu_1 - \mu_2 = 13 $,
- $ H_1: \mu_1 - \mu_2 \neq 13 $.


In [35]:
x1, x2 = 61, 49 # Средние выборок
m1,m2 = 13, 0  # Средние генеральных совокупностей
n1, n2 = 70, 70 # Размеры выборок
s1,s2 = 5, 5 # Стандартные отклонения выборок

In [36]:
t_stat_duo(x1, x2, m1, m2, n1, n2, s1, s2, alpha = 0.05)

Результаты t-теста:
t-статистика: -1.183
Ошибка стандартной оценки (ESE): 0.845
Степени свободы (df): 138.000
p-значение: 0.23876
Решение: Не отвергаем нулевую гипотезу


Соответствующее значение p-value = 0.24, что больше 0.05, а значит, нет оснований утверждать, что технология производства нарушается. Отличий между выборками нет.

**Задача**


Регулярно покупая смузи в магазине у дома, вы обратили внимание на то, что крышечки от бутылок могут быть либо чёрного, либо красного цвета. Причём наличие двух цветов в крышечек наблюдается как для ягодного смузи, так и для фруктового. Будучи очень большим фанатом смузи и статистики, вы решили выяснить, есть ли разница в доле бутылочек с красной крышечкой для двух видов смузи. В течение года вы записывали информацию о купленных вами бутылочках в блокнот.

Результаты оказались следующими:

- доля ягодных смузи с красной крышечкой: $ \overline{p}_1 = 0.4 $,
- доля фруктовых смузи с красной крышечкой: $ \overline{p}_2 = 0.34 $,
- количество купленных ягодных смузи: $ n_1 = 1053 $,
- количество купленных фруктовых смузи: $ n_2 = 995 $.

Проверьте гипотезу о том, что доля смузи с красной крышечкой одинакова для фруктовых и для ягодных смузи:

- $ H_0: \overline{p}_1 - \overline{p}_2 = 0 $,
- $ H_1: \overline{p}_1 - \overline{p}_2 \neq 0 $.

In [37]:
z_prop_test(0.4, 0.34, 1053, 995)

Результаты z-теста:
z-статистика: 2.818
Ошибка стандартной оценки (ESE): 0.021
p-значение: 0.00484
Решение: Отвергаем нулевую гипотезу


Соответствующее значение p-value 0.004.Это значение ниже уровня значимости. Получается, что доля красных крышечек у ягодных смузи статистически значимо превышает долю красных крышечек у фруктовых

### Использование встроенных функций

Вы работаете аналитиком в компании по производству энергетических напитков. Компания выпускает два вида напитков: "Энерго-Буст" и "Энерго-Макс". Существует предположение, что продолжительность энергетического эффекта у "Энерго-Макс" больше, чем у "Энерго-Буст". Ваша задача — проверить это предположение.

**Данные:**

Вы выбрали случайную выборку из 30 человек, которым дали "Энерго-Буст", и замерили среднюю продолжительность энергетического эффекта, которая составила 4.5 часа, со стандартным отклонением 1.2 часа.
Также вы выбрали случайную выборку из 30 других человек, которым дали "Энерго-Макс", и замерили среднюю продолжительность энергетического эффекта, которая составила 5.1 часа, со стандартным отклонением 1.4 часа.

**Гипотезы:**

<p style="text-indent: 40px;"> Нулевая гипотеза ($H_0$): Средняя продолжительность эффекта "Энерго-Буст" равна средней продолжительности эффекта "Энерго-Макс" ($\mu_{\text{буст}} = \mu_{\text{макс}}$).
<p style="text-indent: 40px;"> Альтернативная гипотеза ($H_1$): Средняя продолжительность эффекта "Энерго-Макс" не равна средней продолжительности эффекта "Энерго-Буст" ($\mu_{\text{макс}} \neq \mu_{\text{буст}}$).

In [38]:
x1, x2 = 4.5, 5.1 # Средние выборок
m1,m2 = 0, 0 # Средние генеральных совокупностей
n1, n2 = 30, 30 # Размеры выборок
s1,s2 = 1.2, 1.4 # Стандартные отклонения выборок

Используем ранее написанную самостоятельно функцию

In [39]:
t_stat_duo(x1, x2, m1, m2, n1, n2, s1, s2)

Результаты t-теста:
t-статистика: -1.782
Ошибка стандартной оценки (ESE): 0.337
Степени свободы (df): 56.674
p-значение: 0.08006
Решение: Не отвергаем нулевую гипотезу


P-значение 0.08006 больше обычно используемого порога в 0.05, поэтому вы не отвергаете нулевую гипотезу. Это означает, что на основе имеющихся у нас данных нет достаточно убедительных доказательств статистически значимого различия между средней продолжительностью эффекта "Энерго-Буст" и "Энерго-Макс"

А теперь попробуем как работает готовая встроенная функция *ttest_ind_from_stats*

In [40]:
# Вычисление t-статистики и p-значения
t_stat, p_value = stats.ttest_ind_from_stats(mean1=x1, std1=s1, nobs1=n1,
                                             mean2=x2, std2=s2, nobs2=n2,
                                             equal_var=False)  # equal_var=False для Welch's t-test

print(f"t-статистика: {t_stat:.3f}")
print(f"p-значение: {p_value:.5f}")

t-статистика: -1.782
p-значение: 0.08006


Получаем идентичные значения для встроенной и самописной функции

Таким образом, можно руководствоваться следующими принципами в выборе теста

**Т-тест**:

 - Используется для сравнения средних двух групп, когда размеры выборок невелики (обычно менее 30).
 - Предполагает, что данные имеют приблизительно нормальное распределение.
 - Подходит для ситуаций, когда дисперсии в группах неизвестны и могут не быть равными.

**Бакетный анализ (Bucket testing)** :

 - Используется для всяких активностей, при этом пользователи разделяются на группы (бакеты) для тестирования изменений или нововведений.
 - Подходит для анализа метрик с нестандартным распределением или когда метрики подвержены сильным вариациям и выбросам.
 - Размер выборки лучше будет большим, поскольку важно обеспечить равномерное распределение между бакетами.

**Z-тест:**

- Применяется для сравнения средних, когда размеры выборок велики (обычно более 30).
- Предполагается, что известны стандартные отклонения генеральных совокупностей или что выборки достаточно велики для нормальной аппроксимации распределения средних.
- Часто используется для сравнения пропорций, например, в тестировании конверсии.

🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿🗿